In [2]:
%%gremlin
g.V().limit(100) //check existing vertices

In [14]:
%%gremlin
g.E().limit(10) //check existing edges

In [15]:
%%gremlin
g.V().valueMap(true) //check other properties in the graph

In [16]:
%%gremlin
g.V().as('a') // lets find the simpliest case: 2 nodes linked to 1 common node
 .out().as('b')
 .out().as('c')
 .filter(__.out().where(eq('a')))
 .select('a', 'b', 'c')
 .dedup()


In [17]:
%%gremlin
g.V().as('a') // now we enforce ordering to remove duplicates
 .out().as('b')
 .out().as('c')
 .filter(__.out().where(eq('a')))
 .where('a', lt('b')) // Ensure 'a' < 'b' to enforce order
 .where('b', lt('c')) // Ensure 'b' < 'c' to enforce order
 .select('a', 'b', 'c')
 .dedup()

In [20]:
%%gremlin                               
g.V().as('a')                          // now lets find closed triangle (loop). Start with vertex 'a'
 .outE().as('ab').inV().as('b')        // Traverse an edge 'ab' from 'a' to 'b'
 .outE().as('bc').inV().as('c')        // Traverse an edge 'bc' from 'b' to 'c'
 .outE().as('ca').inV().where(eq('a')) // Explicitly traverse the edge 'ca' back to 'a'
 .path()                               // Include vertices and edges in the path
 .by('accountId')                      // Use accountId for vertex labels
 .by()                                 // Include full edge objects
 .limit(5)                             // Limit to 5 results for better visibility


In [21]:
%%gremlin                              
g.V().as('a')                          // add condition on transaction amount. it should be same sum transfered. Start with vertex 'a'
 .outE().as('ab').inV().as('b')        // Traverse an edge 'ab' from 'a' to 'b'
 .outE().as('bc').inV().as('c')        // Traverse an edge 'bc' from 'b' to 'c'
 .outE().as('ca').inV().where(eq('a')) // Explicitly traverse the edge 'ca' back to 'a'
 .where(select('ab', 'bc', 'ca')       // Ensure all edge amounts are the same
        .by('amount')
        .fold()
        .dedup()
        .count(local).is(1))           // Check that the amounts are identical
 .path()                               // Include vertices and edges in the path
 .by('accountId')                      // Use accountId for vertex labels
 .by()                                 // Include full edge objects
 .limit(5)                             // Limit to 5 results
